# SNN to detect HFOs in iEEG data

In [216]:
import os, sys
import glob
import zipfile
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

# Adjust the Working Directory

In [217]:
# Print current Working Directory
print(os.getcwd())

# Check if the current WD is the file location
if "lava-dl/src/hfo" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/lava-dl/src/hfo/"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/lava-dl/src/hfo


# Check if GPU is Available

In [218]:
# Check if GPU is available
print(torch.cuda.is_available())

print(torch.cuda.device_count())

False
1


# Create Network

A slayer network definition follows standard PyTorch way using `torch.nn.Module`.

The network can be described with a combination of individual `synapse`, `dendrite`, `neuron` and `axon` components. For rapid and easy development, slayer provides __block interface__ - `slayer.block` - which bundles all these individual components into a single unit. These blocks can be cascaded to build a network easily. The block interface provides additional utilities for normalization (weight and neuron), dropout, gradient monitoring and network export.

In the example below, `slayer.block.cuba` is illustrated.

In [219]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        # TODO: Change the parameters of the neuron
        neuron_params = {
                'threshold'     : 1.25,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,     
        }
        # Add dropout to the network
        neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        
        self.blocks = torch.nn.ModuleList([
                # Input of 2 neurons (spike train UP and DOWN) and output of 256 neurons
                slayer.block.cuba.Dense(neuron_params_drop, 2, 256,
                                        weight_norm=True,
                                        delay=True
                                    ),
                
                # Final Layer with 2 neurons that spike if a Ripple or Fast Ripple are detected respectively
                slayer.block.cuba.Dense(neuron_params, 256, 2, weight_norm=True),
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike
    
    def grad_flow(self, path):
        # helps monitor the gradient flow
        grad = [b.synapse.grad_norm for b in self.blocks if hasattr(b, 'synapse')]

        plt.figure()
        plt.semilogy(grad)
        plt.savefig(path + 'gradFlow.png')
        plt.close()

        return grad

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

# Instantiate Network, Optimizer, DataSet and DataLoader

Running the network in GPU is as simple as selecting `torch.device('cuda')`.

In [220]:
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

device = torch.device('cpu')
# device = torch.device('cuda') 

net = Network().to(device)
# net.double()
# net.float()

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

## Prepare the Dataset and DataLoader

### Import the Dataset to a numpy array

Since the `lava-dl` package has a symbolic link to the `lava` package, we can use the `utils` module from the `lava` package directly

In [221]:
# Example of how to read input spikes
from utils.io import preview_np_array

input_filename = 'snn_input_ripple_5_-5.npy'
input_spikes = np.load(f"data/{input_filename}")

preview_np_array(input_spikes, "Input Spikes")

Input Spikes Shape: (245760, 2).
Preview: [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


### Define the `Datasets` and `DataLoaders`

`PyTorch` provides 2 data primitives - `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` to work with data.
- `Dataset` stores the samples and their corresponding labels.
- `DataLoader` wraps an iterable around the `Dataset` to enable easy access to the samples.

In [222]:
from torch.utils.data import DataLoader
from data.ripple_spike_trains_dataset import SpikeTrainsDataset

# Load the Dataset by providing the filename
input_filename = 'data/snn_input_ripple_5_-5.npy'
annotations_filename = 'ground_truth/ch-63.npy'
snn_dataset = SpikeTrainsDataset(input_filename, annotations_filename, device=device, verbose=True)

Input Spikes Shape: torch.Size([245760, 2]).
Preview: tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], dtype=torch.float64)
Annotations Shape: torch.Size([245760]).
Preview: tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)


### Split the data into training, validation and test sets

Need to evaluate the need for a validation set. For now, we will use a 70/10/20 split for training, validation and test sets respectively.

The **spike trains data is ordered**, so we **cannot shuffle** the data as it would disrupt the temporal sequence of the data.

In [223]:
from sklearn.model_selection import train_test_split

# Split the dataset into Training and Validation+Testing tests
# 70% Training, 30% Validation+Testing
# Since we are working with temporal data, we will split the dataset in order (NO SHUFFLE)
train_dataset, val_test_dataset = train_test_split(snn_dataset, test_size=0.3, shuffle=False)

# Split the Validation+Testing dataset into Validation and Testing sets
# 10% Validation, 20% Testing
val_dataset, test_dataset = train_test_split(val_test_dataset, test_size=0.67, shuffle=False)

### Create the `DataLoader` objects
The `batch_size` parameter determines the number of samples that will be loaded and passed through the network at once during training. It's a form of stochastic gradient where instead of updating the weights after each sample, you can update the weights based on a subset of data (a batch).

When working with temporal sequential data, the order is very important. However, the `batch_size` does not directly affect the order of the data. In fact, when working with time series data, it might be relevant to use sequences/windows of data as input for the model. For example, if we want to predict the next value of a sequence, we can use a window of `n` samples as input. In this case, each "sample" in your batch would actually be a sequence of `n` values.

Once again, shuffle is set to `False` to maintain the temporal sequence of the data.

### **Batch Size**
The batch size is a hyperparameter that defines the number of samples to work through before updating the internal model parameters. In our case, it makes sense to use a batch size equal to the ***prediction window*** size. This way, we can identify the presence of Ripple or Fast Ripples using the entire window of data.

In [224]:
# Define the batch_size / window_size
window_size = 100

In [225]:
# Create the DataLoader for the Training, Validation and Testing Datasets
train_loader = DataLoader(train_dataset, batch_size=window_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=window_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=window_size, shuffle=False)

# Visualize the input data

A `slayer.io.Event` can be visualized by invoking it's `Event.show()` routine. `Event.anim()` instead returns the event visualization animation which can be embedded in notebook or exported as video/gif. Here, we will export gif animation and visualize it.

To visualize the input data as a gif, we need to convert each time step into an image. We can use the `Event.show()` method to do this. The `Event.show()` method returns a `matplotlib` figure which can be saved as an image. We can then use the `imageio` package to convert the images into a gif.

In [226]:
# Convert each time step into an image
img_height = 25
img_width = 25

spike_width = 3     # Width of the spike
spike_interval = 2  # Space between spikes
spike_positions = [0, 5, 10, 15, 20]    # Positions of the spikes based on spike_width and spike_interval

y_padding = 5
up_spike_y = img_height - y_padding
down_spike_y = y_padding

def color_pixels(batch, img, idx, x_position_idx):
    # Check if batch[idx] had an UP spike
    if batch[idx][0] == 1:
        # Color the pixels where the spike is detected
        img[up_spike_y][spike_positions[x_position_idx]:spike_positions[x_position_idx]+spike_width] = 1
    
    # Check if batch[idx] had a DOWN spike
    if batch[idx][1] == 1:
        # Color the pixels where the spike is detected
        img[down_spike_y][spike_positions[x_position_idx]:spike_positions[x_position_idx]+spike_width] = 1

In [227]:
def batch_to_image_array(batch):
    """
    Convert a batch of spikes into an array of images
    """
    # Create new batch object with the shape (window_size, img_height, img_width)
    new_batch = np.ndarray(shape=(len(batch), img_height, img_width))

    for idx in range(len(batch)):
        # Initialize the image as a 2D array of zeros
        img = np.zeros(shape=(img_height, img_width))

        # Color the middle line to represent the current time step (Width 1)
        img[1:, img_width//2] = 0.5

        # --- Color the pixels where a spike is detected as 1 --- # 
        if idx >= 2:
            # Color the pixels of the t-2 time step
            color_pixels(batch, img, idx-2, 0)
        if idx >= 1:
            # Color the pixels of the t-1 time step
            color_pixels(batch, img, idx-1, 1)
        # Color the pixels of the current time step

        color_pixels(batch, img, idx, 2)
        if idx+1 < len(batch):
            # Color the pixels of the t+1 time step
            color_pixels(batch, img, idx+1, 3)
        if idx+2 < len(batch):
            # Color the pixels of the t+2 time step
            color_pixels(batch, img, idx+2, 4)
        
        # Update the new_batch with the current batch corresponding image
        new_batch[idx] = img
        

    return new_batch

In [228]:
from utils.io import make_gif

skip_iter = 80
max_iter = 10

EXPORT_GIFS = False
if EXPORT_GIFS:
    for (i, batch) in enumerate(train_loader):
        if i < skip_iter:
            continue

        if i - skip_iter >= max_iter:
            break

        # Fetch an input and target from the training dataset
        train_features, train_label = batch

        # print("\nIdx:", i)
        
        # Convert the batch of spikes into an array of images
        batch_img_array = batch_to_image_array(train_features)
        # preview_np_array(batch_img_array, "Image Array")
        
        # Reshape the batch_img_array, adding 5 frames at the start full of zeros to represent the beggining of the gif
        batch_img_array = np.concatenate((np.zeros((5, img_height, img_width)), batch_img_array), axis=0)

        # Create a GIF from the images using Pillow
        make_gif(batch_img_array, f"data/gifs/input-{i}.gif", duration=40, loop=0)

### Display Some of the GIFs representing input examples

Below you can see batches 84 and 85 respectively.


<table>
    <tr>
        <td> <img src="./data/gifs/input-84.gif" width="200"/> </td>
        <td style="border:0;"> </td>
        <td style="border:0;"> </td>
        <td> <img src="./data/gifs/input-85.gif" width="200"/> </td>
    </tr>
</table>

# Convert the Input to a Tensor shape supported by the Slayer Network

In [229]:
def format_batch_data(batch):
    """
    Format the batch data into the correct format for the network
    """
    # print(f"batch length: {len(batch)}")

    # Convert the batch into a tensor
    batch = torch.tensor(batch, dtype=torch.float64)

    # -- Create tensor with 4/5 dimensions (Batch (optional), Channels, Height, Width, Time) --
    # The batch and Channels dimensions already exist in the object
    # Add the Height and Width dimensions
    batch = batch.unsqueeze(2)
    batch = batch.unsqueeze(3)
    # Add the Time dimension
    batch = batch.unsqueeze(4)

    return batch

In [233]:
# Testing the format_batch_data function
train_batch, _ = next(iter(train_loader))
formatted_data = format_batch_data(train_batch)
# preview_np_array(formatted_data, "Formatted Data")

Formatted Data Shape: torch.Size([100, 2, 1, 1, 1]).
Preview: tensor([[[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.]]],


         [[[0.]]]],



        [[[[0.

/tmp/ipykernel_118841/3801305566.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float64)


In [174]:
# Convert the input to a tensor shape supported by Slayer
# It is assumed that all the tensors that are being processed are 5 dimensional. 
# (Batch, Channels, Height, Width, Time) or NCHWT format. 
# The user must make sure that an input of correct dimension is supplied.
# TODO: I guess the time must be fed to the model. IT can be calculated in hte SpikeTrainsDataset as long as it knows the sampling rate.
input, target = train_dataset[0]
# preview_np_array(input, "Input")

# -- Create tensor with 4/5 dimensions (Batch (optional), Channels, Height, Width, Time) --
# The channels dimension is already included in the original input (2 channels)
# Add the height and width dimensions
formatted_input = torch.unsqueeze(input, 1)
formatted_input = torch.unsqueeze(formatted_input, 2)
# Add the time dimension
formatted_input = torch.unsqueeze(formatted_input, 3)
preview_np_array(formatted_input, "Formatted Input")

# Reformat the target tensor as well
# Add the channel dimension
formatted_target = torch.unsqueeze(target, 0)
# Add the height and width dimensions
formatted_target = torch.unsqueeze(formatted_target, 1)
formatted_target = torch.unsqueeze(formatted_target, 2)
# Add the time dimension
formatted_target = torch.unsqueeze(formatted_target, 3)
preview_np_array(formatted_target, "Formatted Target")

Formatted Input Shape: torch.Size([2, 1, 1, 1]).
Preview: tensor([[[[0.]]],


        [[[0.]]]], dtype=torch.float64)
Formatted Target Shape: torch.Size([1, 1, 1, 1]).
Preview: tensor([[[[0.]]]], dtype=torch.float64)


# Error module

Slayer provides prebuilt loss modules: `slayer.loss.{SpikeTime, SpikeRate, SpikeMax}`.
* `SpikeTime`: precise spike time based loss when target spike train is known.
* `SpikeRate`: spike rate based loss when desired rate of the output neuron is known.
* `SpikeMax`: negative log likelihood losses for classification without any rate tuning.


Since the target spike train $\hat{\boldsymbol s}(t)$ is known for this problem, we use `SpikeTime` loss here. It uses _van Rossum_ like spike train distance metric. The actual and target spike trains are filtered using a FIR filter and the norm of the timeseries is the loss metric.

$$L = \frac{1}{2T} \int_T \left(h_\text{FIR} * ({\boldsymbol s} - \hat{\boldsymbol s})\right)(t)^\top{\bf 1}\,\text dt $$

* `time_constant`: time constant of the FIR filter.
* `filter_order`: the order of FIR filter. Exponential decay is first order filter.

In [175]:
# Define the loss function
# TODO: Check values of the arguments
# TODO: Could be another loss function? SpikeMoid?

error = slayer.loss.SpikeMax(
    moving_window=window_size,
    # time_constant=2, 
    # filter_order=2
).to(device)

### Spike Time Loss Illustration

In [176]:
net_input = formatted_input.to(device)
preview_np_array(net_input, "Net Input")

Net Input Shape: torch.Size([2, 1, 1, 1]).
Preview: tensor([[[[0.]]],


        [[[0.]]]], dtype=torch.float64)


In [177]:
# The followng portion just illustrates the SpikeTime loss calculation. 
# IT IS NOT NEEDED IN PRACTICE

# The network expects a 4D or 5D tensor as input (Batch (optional), Channels, Height, Width, Time)
net_input = formatted_input.to(device)

output = net(net_input)[0]

# just considering first neuron for illustration
output_trace = error.forward(output[0].to(device), formatted_target).flatten().cpu().data.numpy()
target_trace = error.forward(target.to(device), output).flatten().cpu().data.numpy()
fig, ax = plt.subplots(2, 1, figsize=(15, 3), sharex=True)
ax[0].plot(output_trace, label='output trace')
ax[0].plot(target_trace, label='target trace')
ax[1].plot(output_trace - target_trace, label='error trace')
ax[0].set_ylabel('trace')
ax[1].set_ylabel('trace')
ax[1].set_xlabel('time [ms]')
for a in ax: a.legend()

RuntimeError: expected scalar type Double but found Float

# Stats and Assistants

Slayer provides `slayer.utils.LearningStats` as a simple learning statistics logger for training, validation and testing.

In addtion, `slayer.utils.Assistant` module wraps common training validation and testing routine which help simplify the training routine.

In [178]:
stats = slayer.utils.LearningStats()

# Not specifying a Classifier. Which means we are using Regression mode (default)
assistant = slayer.utils.Assistant(net, error, optimizer, stats,
                                   classifier=slayer.classifier.MovingWindow(window_size)
                                )

# Training Loop

Training loop mainly consists of looping over epochs and calling `assistant.train` utility to train.

* `stats` can be used in print statement to get formatted stats printout.
* `stats.training.best_loss` can be used to find out if the current iteration has the best loss. Here, we use it to save the best model.
* `stats.testing.best_accuracy` can be used to find out if the current iteration has the best testing accuracy.
* `stats.update()` updates the stats collected for the epoch.
* `stats.save` saves the stats in files.

In [235]:
epochs = 1   # Number of epochs

# TODO: Check the training loop implementation

for epoch in range(epochs):
    for i, (input, target) in enumerate(train_loader): # training loop
        formatted_input = format_batch_data(input)

        output = assistant.train(formatted_input, target)
        print(f"Training batch {i}/{len(train_loader)}", end="\r")
        
    print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')
    
    if stats.training.best_loss:
        torch.save(net.state_dict(), trained_folder + '/network.pt')
        
    stats.update()
    stats.save(trained_folder + '/')

/tmp/ipykernel_118841/3801305566.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float64)


AssertionError: Input tensor shape (torch.Size([100, 256, 1, 1, 1])) does not match with Neuron shape (torch.Size([1, 1])).

# Plot the learning curves

Plotting the learning curves is as easy as calling `stats.plot()`.

In [ ]:
stats.plot(figsize=(15, 5))

# Export the best model

Load the best model during training and export it as hdf5 network. It is supported by `lava.lib.dl.netx` to automatically load the network as a lava process.

In [ ]:
if torch.cuda.is_available():
    net.load_state_dict(torch.load(trained_folder + '/network.pt'))
else:
    net.load_state_dict(torch.load(trained_folder + '/network.pt', map_location=torch.device('cpu')))


net.export_hdf5(trained_folder + '/network.net')

# Visualize the network output

Here, we will use `slayer.io.tensor_to_event` method to convert the torch output spike tensor into `slayer.io.Event` object and visualize a few input and output event pairs.

In [ ]:
output = net(input.to(device))
for i in range(5):
    # Reshape the input to the shape (sign_event, height_img, width_img, num_time_bins) to separate each DVS event
    inp_event = slayer.io.tensor_to_event(input[i].cpu().data.numpy().reshape(2, 34, 34, -1))

    # Reshape the output to a list containing the prediction percentage of each class
    out_event = slayer.io.tensor_to_event(output[i].cpu().data.numpy().reshape(1, 10, -1))

    inp_anim = inp_event.anim(plt.figure(figsize=(5, 5)), frame_rate=240)
    out_anim = out_event.anim(plt.figure(figsize=(10, 5)), frame_rate=240)
    inp_anim.save(f'gifs/inp{i}.gif', animation.PillowWriter(fps=24), dpi=300)
    out_anim.save(f'gifs/out{i}.gif', animation.PillowWriter(fps=24), dpi=300)


In [ ]:
html = '<table>'
html += '<tr><td align="center"><b>Input</b></td><td><b>Output</b></td></tr>'
for i in range(5):
    html += '<tr>'
    html += gif_td(f'gifs/inp{i}.gif')
    html += gif_td(f'gifs/out{i}.gif')
    html += '</tr>'
html += '</tr></table>'
display.HTML(html)